In [ ]:
!pip install easyocr ultralytics

In [ ]:
import easyocr
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import cv2

In [ ]:
# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

In [ ]:
# Initialize the YOLO model
detectionmodel = YOLO('yolov8n.pt')

In [ ]:
def read_text_from_image(image_path):
    """
    Takes an image path and returns the text detected in the image.

    Args:
        image_path (str): Path to the input image.

    Returns:
        str: The detected text in the image.
    """
    image = cv2.imread(image_path)
    output = reader.readtext(image)
    result = ""
    for i in range(len(output)):
        result += output[i][1] + " "
    return result

def get_text_coordinates(image_path):
    """
    Takes an image path and returns the coordinates of the detected text.

    Args:
        image_path (str): Path to the input image.

    Returns:
        list: A list of lists, where each inner list represents the coordinates of a detected text box.
    """
    image = cv2.imread(image_path)
    output = reader.readtext(image)
    result = []
    for i in range(len(output)):
        result.append(output[i][0])
    return result

def get_object_coordinates(image_path):
    """
    Takes an image path and returns the coordinates of the detected objects.

    Args:
        image_path (str): Path to the input image.

    Returns:
        list: A list of lists, where each inner list represents the coordinates of a detected object.
    """
    detection_results = detectionmodel(image_path, save=True)
    detection_boxes = detection_results[0].boxes
    classes = detection_results[0].boxes.cls.cpu().numpy()
    detectionnames = detection_results[0].names
    res = detection_boxes.cls.cpu().numpy().tolist()
    cls = []
    for i in res:
        cls.append(detectionnames[i])
    conf = detection_results[0].boxes.conf.cpu().numpy()
    xyxy = detection_results[0].boxes.xyxy.cpu().numpy()
    obj_boxes = []
    for i in range(len(classes)):
        if conf[i] > 0.5:
            x_min = int(xyxy[i][0])
            y_min = int(xyxy[i][1])
            x_max = int(xyxy[i][2])
            y_max = int(xyxy[i][3])
            obj_boxes.append([[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]])

    return obj_boxes,cls

def calculate_area(box):
    """
    Calculates the area of a bounding box.

    Args:
        box (list): A list of four coordinates representing the bounding box.

    Returns:
        float: The area of the bounding box.
    """
    x = [point[0] for point in box]
    y = [point[1] for point in box]
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

def calculate_percentage(area, total_area):
    """
    Calculates the percentage of an area relative to a total area.

    Args:
        area (float): The area to calculate the percentage for.
        total_area (float): The total area.

    Returns:
        float: The percentage of the area relative to the total area.
    """
    return (area / total_area) * 100

def calculate_text_percentage(image_path):
    """
    Calculates the percentage of the image area covered by the detected text.

    Args:
        image_path (str): Path to the input image.

    Returns:
        float: The percentage of the image area covered by the detected text.
    """
    image = cv2.imread(image_path)
    total_image_area = image.shape[0] * image.shape[1]

    boxes = get_text_coordinates(image_path)
    total_boxes_area = 0
    for box in boxes:
        box_area = calculate_area(box)
        total_boxes_area += box_area

    text_percentage = calculate_percentage(total_boxes_area, total_image_area)
    return text_percentage

def calculate_object_percentage(image_path):
    """
    Calculates the percentage of the image area covered by the detected objects.

    Args:
        image_path (str): Path to the input image.

    Returns:
        float: The percentage of the image area covered by the detected objects.
    """
    image = cv2.imread(image_path)
    total_image_area = image.shape[0] * image.shape[1]

    obj_boxes, classes = get_object_coordinates(image_path)
    total_boxes_area = 0
    for box in obj_boxes:
        box_coords = box
        box_area = calculate_area(box_coords)
        total_boxes_area += box_area

    object_percentage = calculate_percentage(total_boxes_area, total_image_area)
    return object_percentage

def calculate_intersection_area(box_a, box_b):
    """Calculates the intersection area between two bounding boxes.

    Args:
    box_a (list of lists): The coordinates of the first bounding box.
    box_b (list of lists): The coordinates of the second bounding box.

    Returns:
    float: The intersection area between the two bounding boxes.
    """
    # Extract coordinates of box_a
    x_min_a = min(coord[0] for coord in box_a)
    y_min_a = min(coord[1] for coord in box_a)
    x_max_a = max(coord[0] for coord in box_a)
    y_max_a = max(coord[1] for coord in box_a)

    # Extract coordinates of box_b
    x_min_b = min(coord[0] for coord in box_b)
    y_min_b = min(coord[1] for coord in box_b)
    x_max_b = max(coord[0] for coord in box_b)
    y_max_b = max(coord[1] for coord in box_b)

    # Calculate intersection coordinates
    x_min_i = max(x_min_a, x_min_b)
    y_min_i = max(y_min_a, y_min_b)
    x_max_i = min(x_max_a, x_max_b)
    y_max_i = min(y_max_a, y_max_b)

    # Calculate intersection area
    intersection_width = max(0, x_max_i - x_min_i + 1)
    intersection_height = max(0, y_max_i - y_min_i + 1)
    intersection_area = intersection_width * intersection_height

    return intersection_area


def calculate_intersection_percentage(image_path):
    """
    Calculates the percentage of the image area covered by the intersection between detected text and objects.

    Args:
        image_path (str): Path to the input image.

    Returns:
        float: The percentage of the image area covered by the intersection between detected text and objects.
    """
    image = cv2.imread(image_path)
    total_image_area = image.shape[0] * image.shape[1]

    text_boxes = get_text_coordinates(image_path)
    obj_boxes, classes = get_object_coordinates(image_path)

    total_intersection_area = 0
    for text_box in text_boxes:
        for obj_box in obj_boxes:

            intersection_area = calculate_intersection_area(text_box, obj_box)
            total_intersection_area += intersection_area

    intersection_percentage = calculate_percentage(total_intersection_area, total_image_area)
    return intersection_percentage

In [ ]:
image_path = "01278.png"

In [ ]:
# Example usage of read_text_from_image function
text_detected = read_text_from_image(image_path)

# Example usage of get_text_coordinates function
text_coordinates = get_text_coordinates(image_path)

# Example usage of get_object_coordinates function
object_coordinates, classes = get_object_coordinates(image_path)

# Example usage of calculate_text_percentage function
text_percentage = calculate_text_percentage(image_path)

# Example usage of calculate_object_percentage function
object_percentage = calculate_object_percentage(image_path)

# Example usage of calculate_intersection_percentage function
intersection_percentage = calculate_intersection_percentage(image_path)


image 1/1 /content/01278.png: 640x320 1 person, 1 tennis racket, 148.1ms
Speed: 4.5ms preprocess, 148.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 320)
Results saved to runs/detect/predict
person
tennis racket

image 1/1 /content/01278.png: 640x320 1 person, 1 tennis racket, 122.5ms
Speed: 3.1ms preprocess, 122.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 320)
Results saved to runs/detect/predict
person
tennis racket

image 1/1 /content/01278.png: 640x320 1 person, 1 tennis racket, 393.3ms
Speed: 3.1ms preprocess, 393.3ms inference, 13.7ms postprocess per image at shape (1, 3, 640, 320)
Results saved to runs/detect/predict
person
tennis racket


In [ ]:
print("Detected Text:", text_detected)
print("Text Coordinates:", text_coordinates)
print("Object Detected:", classes)
print("Object Coordinates:", object_coordinates)
print("Text Percentage:", text_percentage)
print("Object Percentage:", object_percentage)
print("Intersection Percentage:", intersection_percentage)

Detected Text: hope on a rope 1 ilsHhaleezueshbbl waiting for 
Text Coordinates: [[[60, 0], [429, 0], [429, 59], [60, 59]], [[345, 361], [359, 361], [359, 387], [345, 387]], [[33, 345], [479, 345], [479, 455], [33, 455]], [[77, 979], [406, 979], [406, 1060], [77, 1060]]]
Object Detected: ['person', 'tennis racket']
Object Coordinates: [[[60, 1], [511, 1], [511, 332], [60, 332]]]
Text Percentage: 17.776889534883722
Object Percentage: 27.122274709302324
Intersection Percentage: 3.966206395348837
